In [1]:
import os
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from haversine import haversine, Unit
from sklearn.metrics.pairwise import haversine_distances
from tqdm import tqdm
from matplotlib.colors import ListedColormap, BoundaryNorm, LogNorm
import pandas as pd
from xhistogram.xarray import histogram
from sklearn.neighbors import KNeighborsRegressor, BallTree
from scipy import stats
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cmocean

def haversine_sklearn(lat1, lon1, lat_array, lon_array):


    X = np.array([lat1, lon1]).reshape(1, -1)
    Y = np.array([lat_array, lon_array]).T
    X = np.radians(X)
    Y = np.radians(Y)

    return haversine_distances(X, Y) * 6357
from matplotlib.colors import BoundaryNorm

colors = np.array([[0.10588235, 0.61960784, 0.46666667, 1.],
                [0.45882353, 0.43921569, 0.70196078, 1.],
                [0.4       , 0.4       , 0.4       , 1.],
                [0.90196078, 0.67058824, 0.00784314, 1.],
])

cmap = ListedColormap(colors)
#cmap = plt.get_cmap('Dark2')
bounds = [1, 2, 3, 4, 5]
norm = BoundaryNorm(bounds, cmap.N)
# Create a figure and axis with the specified size

SIC = xr.open_dataset('/projekt_agmwend/home_rad/Joshua/HALO-AC3_unified_data/amsr_modis_sic.nc').sortby('time')


fontsize = 8

plt.rcParams['font.size'] = fontsize
plt.rcParams['axes.labelsize']  = fontsize
plt.rcParams['axes.titlesize']  = fontsize
plt.rcParams['xtick.labelsize'] = fontsize
plt.rcParams['ytick.labelsize'] = fontsize
plt.rcParams['legend.fontsize'] = fontsize
plt.rcParams['legend.title_fontsize'] = fontsize
plt.rcParams['figure.titlesize'] = fontsize
plt.rcParams['figure.dpi'] = 300
plt.rcParams['axes.axisbelow'] = True
sc_cmap = 'Dark2'


cmap = ListedColormap(colors)


ERROR 1: PROJ: proj_create_from_database: Open of /home/jomueller/micromamba/envs/mamba_josh/share/proj failed


In [2]:
outfiles_keys = [f.split('_full')[0] for f in os.listdir(f'/projekt_agmwend/home_rad/Joshua/Mueller_et_al_2024/data/full_datasets/') if 'full' in f and 'T' in f]

outfiles_keys.sort()

outfiles = os.listdir('/projekt_agmwend/home_rad/Joshua/Mueller_et_al_2024/data/full_datasets/')
outfiles = [f'/projekt_agmwend/home_rad/Joshua/Mueller_et_al_2024/data/full_datasets/{f}' for f in outfiles if 'full' in f and 'T' in f]
outfiles.sort() 

segmentation_path = '/projekt_agmwend/home_rad/Joshua/Mueller_et_al_2024/data/segmented_data_nadir/'
segmentation_files = [os.path.join(segmentation_path, f) for f in os.listdir(segmentation_path)]
segmentation_files.sort()

### compare the length
len_outfiles = len(outfiles)
len_segmentation_files = len(segmentation_files)

if len_outfiles != len_segmentation_files:
    print('The length of the outfiles and the segmentation files does not match')
    print(f'len_outfiles: {len_outfiles}')
    print(f'len_segmentation_files: {len_segmentation_files}')


df_files = pd.DataFrame(columns=['outfile', 'segfile', 'key'])
outfiles_df = []
segfiles_df = []
keys_df = []

for i in range(min([len_outfiles, len_segmentation_files])):
    key = outfiles_keys[i]
    outfile = outfiles[i]
    segfile = [f for f in segmentation_files if key in f][0]
    # print(f'Working on {key}')
    # print(f'Outfile: {outfile}')
    # print(f'Segfile: {segfile}')

    outfiles_df.append(outfile)
    segfiles_df.append(segfile)
    keys_df.append(key)

df_files['outfile'] = outfiles_df
df_files['segfile'] = segfiles_df
df_files['key'] = keys_df

df_files

,outfile,segfile,key
0,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,2022-03-21T11:39:00_2022-03-21T11:44:00
1,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,2022-03-28T10:28:00_2022-03-28T11:05:00
2,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,2022-03-28T13:13:30_2022-03-28T13:21:30
3,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,2022-03-28T14:12:30_2022-03-28T14:27:00
4,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,2022-03-29T14:27:00_2022-03-29T14:37:30
5,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,2022-03-30T09:46:30_2022-03-30T09:52:00
6,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,2022-04-01T09:25:00_2022-04-01T09:32:30
7,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,2022-04-01T11:22:30_2022-04-01T11:43:30
8,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,2022-04-01T12:20:00_2022-04-01T12:51:00
9,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,2022-04-01T13:36:00_2022-04-01T13:52:30


In [3]:
df_files.to_csv('../../../data/final_filelist_to_submission.csv', index=False)

In [4]:
outfiles_keys = [f.split('_full')[0] for f in os.listdir(f'../../../data/cluster/output_sam/') if 'full' in f and 'T' in f]

outfiles_keys.sort()

outfiles = os.listdir('../../../data/cluster/output_sam')
outfiles = [f'../../../data/cluster/output_sam/{f}' for f in outfiles if 'full' in f and 'T' in f]
outfiles.sort() 

segmentation_path = '/projekt_agmwend/home_rad/Joshua/Mueller_et_al_2024/data/segmented_data/'
segmentation_files = [os.path.join(segmentation_path, f) for f in os.listdir(segmentation_path)]
segmentation_files.sort()

### compare the length
len_outfiles = len(outfiles)
len_segmentation_files = len(segmentation_files)

if len_outfiles != len_segmentation_files:
    print('The length of the outfiles and the segmentation files does not match')
    print(f'len_outfiles: {len_outfiles}')
    print(f'len_segmentation_files: {len_segmentation_files}')


df_files = pd.DataFrame(columns=['outfile', 'segfile', 'key'])
outfiles_df = []
segfiles_df = []
keys_df = []

for i in range(min([len_outfiles, len_segmentation_files])):
    key = outfiles_keys[i]
    outfile = outfiles[i]
    segfile = [f for f in segmentation_files if key in f][0]
    # print(f'Working on {key}')
    # print(f'Outfile: {outfile}')
    # print(f'Segfile: {segfile}')

    outfiles_df.append(outfile)
    segfiles_df.append(segfile)
    keys_df.append(key)

df_files['outfile'] = outfiles_df
df_files['segfile'] = segfiles_df
df_files['key'] = keys_df

df_files


The length of the outfiles and the segmentation files does not match
len_outfiles: 16
len_segmentation_files: 17


,outfile,segfile,key
0,../../../data/cluster/output_sam/2022-03-20T10...,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,2022-03-20T10:35:00_2022-03-20T10:50:00
1,../../../data/cluster/output_sam/2022-03-21T11...,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,2022-03-21T11:39:00_2022-03-21T11:44:00
2,../../../data/cluster/output_sam/2022-03-28T10...,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,2022-03-28T10:28:00_2022-03-28T11:05:00
3,../../../data/cluster/output_sam/2022-03-28T13...,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,2022-03-28T13:13:30_2022-03-28T13:21:30
4,../../../data/cluster/output_sam/2022-03-28T14...,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,2022-03-28T14:12:30_2022-03-28T14:27:00
5,../../../data/cluster/output_sam/2022-03-29T14...,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,2022-03-29T14:27:00_2022-03-29T14:37:30
6,../../../data/cluster/output_sam/2022-03-30T09...,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,2022-03-30T09:46:30_2022-03-30T09:52:00
7,../../../data/cluster/output_sam/2022-04-01T09...,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,2022-04-01T09:25:00_2022-04-01T09:32:30
8,../../../data/cluster/output_sam/2022-04-01T10...,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,2022-04-01T10:21:00_2022-04-01T10:54:00
9,../../../data/cluster/output_sam/2022-04-01T11...,/projekt_agmwend/home_rad/Joshua/Mueller_et_al...,2022-04-01T11:22:30_2022-04-01T11:43:30


In [5]:
ds_list = []
ds_full_list = []


for index, row in tqdm(df_files.iterrows(), total=df_files.shape[0]):
    full_file = row['outfile']
    segment_file = row['segfile']

    ds_full = xr.open_dataset(full_file)
    ds_full['lat'] = ds_full['lat'].where(ds_full['lat'] != 0)
    ds_full['lon'] = ds_full['lon'].where(ds_full['lon'] != 0)
    ds_segment = xr.open_dataset(segment_file)
    start_time = full_file.split('/')[-1].split('.nc')[0].split('_')[0]
    end_time = full_file.split('/')[-1].split('.nc')[0].split('_')[1]
    date_range = pd.date_range(start=start_time, end=end_time, periods=ds_full.x.size)
    date = pd.to_datetime(start_time).strftime('%Y-%m-%d')
    if date == '2022-03-20' or date == '2022-03-21':
        continue
    N = ds_segment['segment'].size
    date_array = np.array([date for _ in range(N)])

    ds_segment['segment_date'] = ('segment', date_array)
    ds_full['time'] = ('x', date_range)

    h = histogram(ds_full['smoothed_prediction'].isel(y=slice(635//2-150, 635//2+150)), bins=[[.5, 1.5, 2.5, 3.5, 4.5]], dim=['y'])

    track_lat = ds_full['lat'].mean(dim='y')#isel(y=slice(635//2-150, 635//2+150))
    track_lon = ds_full['lon'].mean(dim='y')#isel(y=slice(635//2-150, 635//2+150))

    ### fill the nan values with the mean between the two nearest values

    track_lat = track_lat.where(~track_lat.isnull(), other=0)
    track_lon = track_lon.where(~track_lon.isnull(), other=0)

    print(date)

    modis_amsr_sic = SIC.sel(time=date)['z']

    modis_amsr_sic_sel = modis_amsr_sic.sel(
        x=track_lon.values,
        y=track_lat.values,
        method='nearest'
    )

    ds_sic = xr.Dataset(
        data_vars=dict(
            counts=(('time', 'class_bin'), h.values),
            sic=(('time'), np.diag(modis_amsr_sic_sel.values)),
            #distance=(('time'), dist),
        ),
        coords=dict(
            time=ds_full['time'].values,
            class_bin=[1, 2, 3, 4],
        )
    )

    ds_full_list.append(ds_sic)
    ds_list.append(ds_segment)
    

ds = xr.concat(ds_list, dim='segment')
ds_sic = xr.concat(ds_full_list, dim='time')

  0%|          | 0/16 [00:00<?, ?it/s]

 12%|█▎        | 2/16 [00:02<00:21,  1.52s/it]

2022-03-28


 19%|█▉        | 3/16 [00:21<02:02,  9.41s/it]

2022-03-28


...

 88%|████████▊ | 14/16 [01:19<00:14,  7.36s/it]

2022-04-04


 94%|█████████▍| 15/16 [01:23<00:06,  6.60s/it]

2022-04-04


100%|██████████| 16/16 [01:29<00:00,  5.57s/it]


In [11]:
ds['segment_edge_dist'] = xr.DataArray(np.zeros(ds.segment_lat.size) * np.nan, dims=('segment'))
ds['segment_sic'] = xr.DataArray(np.zeros(ds.segment_lat.size) * np.nan, dims=('segment'))
segment_edge_dist = np.zeros(ds.segment_lat.size)  ### in km
selected_edge_points = np.zeros((ds.segment_lat.size, 2))

ds_miz_edge_full = SIC.where((SIC['z'] > 9) & (SIC['z'] < 11)).stack(feature=('time','x', 'y')).dropna('feature')
#ds_miz_edge = SIC.where((SIC['z'] > 9) & (SIC['z'] < 11)).mean(dim='time').stack(feature=('x', 'y')).dropna('feature')

for i, seg in tqdm(enumerate(ds.segment), total=ds.segment.size):
    # if ds.segment_size.isel(segment=i) < 100:
    #     continue
    lat = ds.segment_lat.isel(segment=i)
    lon = ds.segment_lon.isel(segment=i)

    date = ds.segment_date.isel(segment=i)#.dt.strftime('%Y-%m-%d')
    ds_miz_edge = ds_miz_edge_full.sel(time=date)

    miz_lats = ds_miz_edge.y.values
    miz_lons = ds_miz_edge.x.values

    if np.isnan(lat) or np.isnan(lon):
        continue

    haversine_dist = haversine_sklearn(lat, lon, miz_lats, miz_lons).flatten()
    
    min_dist_index = haversine_dist.argmin()

    min_dist_lon = miz_lons[min_dist_index]
    min_dist_lat = miz_lats[min_dist_index]

    edge_dist = haversine_dist[min_dist_index]

    ds['segment_edge_dist'][i] = edge_dist
    ds['segment_sic'][i] = SIC.sel(x=lon, method='nearest').sel(y=lat, method='nearest').sel(time=date).z.values
    segment_edge_dist[i] = edge_dist

    selected_edge_points[i, 0] = min_dist_lon
    selected_edge_points[i, 1] = min_dist_lat

ds['segment_edge_dist'] = xr.DataArray(segment_edge_dist, dims=('segment')) 
ds.to_netcdf('../../../data/ar/HALO-AC3_HALO_VELOX_segmentation_statistics_with_sea_ice_edge_distance.nc', mode='w')

100%|██████████| 37934/37934 [02:05<00:00, 302.67it/s]


In [8]:
ds_sic.to_netcdf('../../../data/ar/HALO-AC3_HALO_VELOX_sea_ice_concentration.nc')

In [10]:
ds.to_netcdf('../../../data/ar/HALO-AC3_HALO_VELOX_segmentation_statistics.nc')